In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e1/sample_submission.csv
/kaggle/input/playground-series-s4e1/train.csv
/kaggle/input/playground-series-s4e1/test.csv


# ****Importing Libraries****

In [2]:
#importing python libraries
import pandas as pd
import pylab as pl
import numpy as np
import scipy.optimize as opt
from sklearn import preprocessing
%matplotlib inline 
import matplotlib.pyplot as plt


# ****Loading the Data****

In [3]:
#loading the data
train_data = pd.read_csv('/kaggle/input/playground-series-s4e1/train.csv')
train_data.head()

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,0,15674932,Okwudilichukwu,668,France,Male,33.0,3,0.00,2,1.0,0.0,181449.97,0
1,1,15749177,Okwudiliolisa,627,France,Male,33.0,1,0.00,2,1.0,1.0,49503.50,0
2,2,15694510,Hsueh,678,France,Male,40.0,10,0.00,2,1.0,0.0,184866.69,0
3,3,15741417,Kao,581,France,Male,34.0,2,148882.54,1,1.0,1.0,84560.88,0
4,4,15766172,Chiemenam,716,Spain,Male,33.0,5,0.00,2,1.0,1.0,15068.83,0


# Exploring Data

In [4]:
# finding null values
train_data.isna().sum()

id                 0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [5]:
#checking data types of columns
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 165034 entries, 0 to 165033
Data columns (total 14 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   id               165034 non-null  int64  
 1   CustomerId       165034 non-null  int64  
 2   Surname          165034 non-null  object 
 3   CreditScore      165034 non-null  int64  
 4   Geography        165034 non-null  object 
 5   Gender           165034 non-null  object 
 6   Age              165034 non-null  float64
 7   Tenure           165034 non-null  int64  
 8   Balance          165034 non-null  float64
 9   NumOfProducts    165034 non-null  int64  
 10  HasCrCard        165034 non-null  float64
 11  IsActiveMember   165034 non-null  float64
 12  EstimatedSalary  165034 non-null  float64
 13  Exited           165034 non-null  int64  
dtypes: float64(5), int64(6), object(3)
memory usage: 17.6+ MB


# Preprocessing The Data

In [6]:
# One-hot encode categorical columns 'Geography' and 'Gender' to numerical features
train_data = pd.get_dummies(train_data, columns= ['Geography','Gender'])
train_data = train_data.replace({True: 1, False: 0})


/tmp/ipykernel_18/3602624293.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_data = train_data.replace({True: 1, False: 0})


In [7]:
train_data.dtypes

id                     int64
CustomerId             int64
Surname               object
CreditScore            int64
Age                  float64
Tenure                 int64
Balance              float64
NumOfProducts          int64
HasCrCard            float64
IsActiveMember       float64
EstimatedSalary      float64
Exited                 int64
Geography_France       int64
Geography_Germany      int64
Geography_Spain        int64
Gender_Female          int64
Gender_Male            int64
dtype: object

In [8]:
# change data type to int
train_data['Age'] = train_data['Age'].astype(int)
train_data['Balance'] = train_data['Balance'].astype(int)
train_data['HasCrCard'] = train_data['HasCrCard'].astype(int)
train_data['IsActiveMember'] = train_data['IsActiveMember'].astype(int)
train_data['EstimatedSalary'] = train_data['EstimatedSalary'].astype(int)

In [9]:
train_data.dtypes

id                    int64
CustomerId            int64
Surname              object
CreditScore           int64
Age                   int64
Tenure                int64
Balance               int64
NumOfProducts         int64
HasCrCard             int64
IsActiveMember        int64
EstimatedSalary       int64
Exited                int64
Geography_France      int64
Geography_Germany     int64
Geography_Spain       int64
Gender_Female         int64
Gender_Male           int64
dtype: object

In [10]:
# creates a feature  'X' with specific attributes from the 'train_data' dataset .
X = train_data[['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember',
            'EstimatedSalary', 'Geography_Germany', 'Geography_Spain', 'Geography_France','Gender_Male','Gender_Female']]

In [11]:
X.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain,Geography_France,Gender_Male,Gender_Female
0,668,33,3,0,2,1,0,181449,0,0,1,1,0
1,627,33,1,0,2,1,1,49503,0,0,1,1,0
2,678,40,10,0,2,1,0,184866,0,0,1,1,0
3,581,34,2,148882,1,1,1,84560,0,0,1,1,0
4,716,33,5,0,2,1,1,15068,0,1,0,1,0


# Training Model

In [12]:
#training target dataset
Y = train_data[['Exited']]
Y.info()




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 165034 entries, 0 to 165033
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype
---  ------  --------------   -----
 0   Exited  165034 non-null  int64
dtypes: int64(1)
memory usage: 1.3 MB


In [13]:
#Split data into training and testing sets for model evaluation.
# X_test_ is the validation dataset 
from sklearn.model_selection import train_test_split
X_train, X_test_, y_train, y_test_ = train_test_split(X, Y, test_size=0.2, random_state=42)

In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier


In [15]:
# fit the training data into Random Forest Classifier Model
model = RandomForestClassifier(random_state=42, max_depth=10, min_samples_leaf=1, min_samples_split=5, n_estimators=200)
model.fit(X_train, y_train)

/tmp/ipykernel_18/872325953.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train, y_train)


RandomForestClassifier(max_depth=10, min_samples_split=5, n_estimators=200,
                       random_state=42)

In [16]:
# Predict the Test Dataset
yhat = model.predict(X_test_)
yhat

array([0, 0, 1, ..., 0, 0, 0])

In [17]:
#Calculate predicted probabilities for positive class (churn) on the Validation set (X_test_) 
yhat_prob = model.predict_proba(X_test_)[:, 1]
yhat_prob

array([0.17926526, 0.02901825, 0.66835209, ..., 0.45541545, 0.02355721,
       0.08071479])

In [18]:
 # Calculate AUC for validation set (X_test_)
from sklearn.metrics import roc_auc_score
roc_auc = roc_auc_score(y_test_,yhat)
print(f'ROC AUC on validation set: {roc_auc}')

ROC AUC on validation set: 0.7366172758724105


In [19]:
# loading and preprocessing test data
test_data = pd.read_csv('/kaggle/input/playground-series-s4e1/test.csv')
test_data.head()
test_data.isna().sum()
test_data = pd.get_dummies(test_data, columns= ['Geography','Gender'])
test_data = test_data.replace({True: 1, False: 0})
test_data.head()

/tmp/ipykernel_18/1856476195.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_data = test_data.replace({True: 1, False: 0})


,id,CustomerId,Surname,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,165034,15773898,Lucchese,586,23.0,2,0.00,2,0.0,1.0,160976.75,1,0,0,1,0
1,165035,15782418,Nott,683,46.0,2,0.00,1,1.0,0.0,72549.27,1,0,0,1,0
2,165036,15807120,K?,656,34.0,7,0.00,2,1.0,0.0,138882.09,1,0,0,1,0
3,165037,15808905,O'Donnell,681,36.0,8,0.00,1,1.0,0.0,113931.57,1,0,0,0,1
4,165038,15607314,Higgins,752,38.0,10,121263.62,1,1.0,0.0,139431.00,0,1,0,0,1


In [20]:
#creates a feature 'X_test' with specific attributes from the 'test_data' dataset .
X_test = test_data[['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember',
            'EstimatedSalary', 'Geography_Germany', 'Geography_Spain', 'Geography_France','Gender_Male','Gender_Female']]


In [21]:
#Calculate predicted probabilities for positive class (churn) on the Test Dataset. 

test_predictions = model.predict_proba(X_test)[:, 1]


In [22]:
test_predictions

array([0.02738389, 0.80913481, 0.04607999, ..., 0.03159565, 0.17347012,
       0.22054399])

In [23]:
submission_df = pd.DataFrame({'id': test_data['id'], 'Exited': test_predictions})
submission_df.to_csv('submission.csv', index=False)

In [24]:
submission_df

,id,Exited
0,165034,0.027384
1,165035,0.809135
2,165036,0.046080
3,165037,0.203421
4,165038,0.322502
...,...,...
110018,275052,0.051100
110019,275053,0.099900
110020,275054,0.031596
110021,275055,0.173470
